# Data Collection

The purpose of this notebook is to gather data from the [Yelp website](https://www.yelp.com/dataset), and convert it into a more usable format. The original dataset is nearly 10GB of JSON files. Since for the scope of this project we are only interested in restaurant data, we are going to convert the files into csv format, and make relevant merges and deletions, in order to make the analysis more memory-efficient. These files will be utilized in the next notebook for exploratory data analysis and modeling.

In [2]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

Yelp Dataset comes with 5 json files: business, checkin, user, review and tip. We convert them into the csv files. for further cleaning and EDA.

**Glossary of df names:** <br>
* **df_business:** yelp's business.json. Contains business_id, name, geo info, stars, review count, attributes, categories and hours. Also exists as *'business_yelp.csv'*.
* **restaurant_df:** created by me, filtered for restaurants that are open. Contains same columns as business, minus the hours. Also exists in csv: *'restaurant_bus_yelp.csv'*.
* **df_user:** yelp's user.json. Not yet filtered for restaurants! Nearly 2 million users listed, so it will have to be merged with other df in order to distinguish restaurants. Also exists as *'user_yelp.csv'*.
* **review.json:** currently *work in progress*. Might not use for this project. It's nearly 6GB. Definitely needs to be reduced and normalized.
* **df_checkin:** yelp's checkin.json. With "total" column added.
* **restaurant_checkin_df** restaurant_df + df_checkin, merged on business_id. Also exists as 'restaurant_checkin.csv'.
* **df_tip:** yelp's tip.json. Contains user_id, business_id, text of the tip, date	and compliment_count.

**Note:** Not all the df's mentioned above will be used in the analysis. In fact, the df that we use for modeling will be created in the EDA notebook. The df's from the glossary are just here for exporatory purposes.

### 1) Business json --> csv

In [30]:
business_json_path = 'yelp_dataset/yelp_academic_dataset_business.json'
df_business = pd.read_json(business_json_path, lines=True)

In [31]:
df_business.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None


In [32]:
df_business.shape

(209393, 14)

In [8]:
df_business.to_csv('business_yelp.csv')

In [33]:
df_business = df_business[df_business['is_open']==1]

In [34]:
drop_columns = ['hours','is_open']
df_business = df_business.drop(drop_columns, axis=1)

#### Restaurant_df created:

More filtering: keeping only the restaurants.

In [35]:
restaurant_df = df_business[df_business['categories'].str.contains(
              'Restaurants',
              case=False, na=False)]

In [36]:
restaurant_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Ethnic Food, Food Trucks, Specialty Food, Impo..."
24,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,"{'RestaurantsTableService': 'False', 'Restaura...","Restaurants, Cheesesteaks, Poutineries"
25,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...","Japanese, Fast Food, Food Court, Restaurants"
30,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,"{'Ambience': '{'touristy': False, 'hipster': F...","Persian/Iranian, Turkish, Middle Eastern, Rest..."
33,vjTVxnsQEZ34XjYNS-XUpA,Wetzel's Pretzels,"4550 East Cactus Rd, #KSFC-4",Phoenix,AZ,85032,33.602822,-111.983533,4.0,10,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...","Food, Pretzels, Bakeries, Fast Food, Restaurants"


In [54]:
restaurant_df.shape

(43965, 12)

In [55]:
df_restaurant_explode = restaurant_df.assign(categories = restaurant_df.categories
                         .str.split(', ')).explode('categories')

In [56]:
df_restaurant_explode.categories.value_counts()

Restaurants                      43965
Food                             11909
Fast Food                         7032
Nightlife                         6455
Bars                              6242
                                 ...  
Trampoline Parks                     1
Foundation Repair                    1
Aircraft Repairs                     1
Hiking                               1
IT Services & Computer Repair        1
Name: categories, Length: 777, dtype: int64

In [57]:
df_restaurant_explode[df_restaurant_explode.categories.str.match('Restaurants',
                      case=True,na=False)].categories.value_counts()

Restaurants    43965
Name: categories, dtype: int64

Now, we will have the csv file with just restaurants:

In [65]:
restaurant_df.to_csv('restaurant_bus_yelp.csv')

### 2) User json --> csv

In [ ]:
user_json_path = 'yelp_dataset/yelp_academic_dataset_user.json'
df_user = pd.read_json(user_json_path, lines=True)

In [10]:
df_user.head(2)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,ntlvfPzc8eglqvk92iDIAw,Rafael,553,2007-07-06 03:27:11,628,225,227,,"oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg...",14,...,2,1,0,1,11,15,22,22,10,0
1,FOBRPlBHa3WPHFB5qYDlVg,Michelle,564,2008-04-28 01:29:25,790,316,400,"2008,2009,2010,2011,2012,2013","ly7EnE8leJmyqyePVYFlug, pRlR63iDytsnnniPb3AOug...",27,...,4,5,2,1,33,37,63,63,21,5


In [11]:
df_user.shape

(1968703, 22)

In [12]:
df_user.to_csv('user_yelp.csv')

Note: I did not end up using the user data for this project, since at this time my approach is product-centric, but I might use this df in further projects down the line, so this dataframe stays here.

### 3) Review json --> csv

In [ ]:
##############################################################################
# I Probaly won't use the review data for this project, but I will need it for NLP!
# Planning to do sentiment analysis of the reviews for the next project.
# Only if I figure out how to drop the text of the reviews and just keep the stars.
##############################################################################

In [5]:
size = 10000
review_json_path = 'yelp_dataset/yelp_academic_dataset_review.json'

# review_df = pd.read_json(review_json_path, lines=True, chunksize=size)
review_df = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [7]:
#review_df = pd.io.json.json_normalize(review_df, max_level = 2)
review_df = pd.DataFrame(review_df)

In [8]:
review_df.head()

""


In [ ]:
##############################################################################
# I Probaly won't use the review data for this project, but I will need it for NLP!
##############################################################################

Note:  I did not end up using the user data for this project, since i am saving it for the NLP. I need to find a better way of handling large files. 

Disclaimer: I advise you against running the code above, it takes forever, and everything crashes.

### 4) Checkins 

In [66]:
checkin_json_path = 'yelp_dataset/yelp_academic_dataset_checkin.json'
df_checkin = pd.read_json(checkin_json_path, lines=True)

In [70]:
df_checkin.sample(10)

,business_id,date
1740,-cFce5TloT-REJDHDgKDbA,"2013-12-22 18:19:55, 2014-01-09 21:17:04, 2014..."
91718,WSpFjl-7g3HpyVb2mqYrVg,"2012-10-17 16:13:42, 2012-10-27 17:13:32, 2012..."
143280,oHrhz4_OBHIr5uPzER8DVA,"2015-05-17 01:52:10, 2015-05-17 01:53:03, 2015..."
173933,zZIhaE3yMCUCJpKsj0oOew,"2010-12-12 22:40:01, 2011-12-30 07:16:53, 2011..."
30767,AEpqgti925fqprXU6Yur7g,2017-01-30 23:13:47
134089,kwEBCIp18Xi9ETDn_WIQaA,"2016-05-14 20:15:13, 2016-05-20 18:52:40, 2016..."
154719,sWSo5pLamuuxc5QMICY-qw,"2012-01-25 21:38:00, 2012-03-27 18:49:46, 2012..."
151389,rGZtr_ZfOVfmj9aTLV11Rw,"2010-07-18 23:04:07, 2010-07-30 21:23:11, 2010..."
10703,2wvykKmGHvwF2GTUZoy9og,"2019-02-05 03:42:01, 2019-02-25 02:44:23, 2019..."
141007,nT5MAzQ2oLwAFp2A44yTCA,"2017-02-04 00:49:16, 2017-04-04 19:57:50"


In [69]:
df_checkin.shape

(175187, 2)

Adding a "total" column to see the number of checkins over time.

In [72]:
df_checkin['total'] = [len(df_checkin.iloc[i,1]) for i in range(0,175187)]

In [73]:
df_checkin.head()

,business_id,date,total
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016...",145
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012...",3967
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015...",4051
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30,19
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010...",796


In [77]:
# Merging checkins and restaurants, so that we see the names of the restaurants.
# We need to perform inner join on restaurant_df and df_checkin.
restaurant_checkin_df = pd.merge(restaurant_df, df_checkin, on='business_id')

In [78]:
restaurant_checkin_df.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,date,total
0,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Ethnic Food, Food Trucks, Specialty Food, Impo...","2013-02-28 19:53:31, 2013-03-14 18:01:50, 2017...",61
1,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,"{'RestaurantsTableService': 'False', 'Restaura...","Restaurants, Cheesesteaks, Poutineries",2019-01-04 01:13:15,19
2,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...","Japanese, Fast Food, Food Court, Restaurants","2014-11-28 18:55:44, 2016-06-12 01:00:30, 2016...",166
3,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,"{'Ambience': '{'touristy': False, 'hipster': F...","Persian/Iranian, Turkish, Middle Eastern, Rest...","2019-03-24 21:23:22, 2019-06-15 22:28:01",40
4,vjTVxnsQEZ34XjYNS-XUpA,Wetzel's Pretzels,"4550 East Cactus Rd, #KSFC-4",Phoenix,AZ,85032,33.602822,-111.983533,4.0,10,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...","Food, Pretzels, Bakeries, Fast Food, Restaurants","2010-07-06 21:38:52, 2010-08-29 17:19:51, 2012...",502
5,98hyK2QEUeI8v2y0AghfZA,Pho Lee's Vietnamese Restaurant,"1541 E 38th St, Ste 101",Cleveland,OH,44114,41.512155,-81.663332,4.5,23,"{'BusinessParking': 'None', 'Ambience': '{'tou...","Restaurants, Vietnamese, Soup","2019-08-18 21:17:31, 2019-08-31 17:06:17, 2019...",271
6,LoRef3ChgZKbxUio-sHgQg,Amir,5252 Rue Jean Talon O,Montréal,QC,H4P 2A7,45.494870,-73.651904,3.0,18,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Restaurants, Lebanese, Middle Eastern","2013-05-22 16:28:19, 2014-06-10 22:04:37, 2014...",208
7,tLpkSwdtqqoXwU0JAGnApw,Wendy's,4602 Northfield Road,Cleveland,OH,44128,41.434614,-81.527026,3.5,7,"{'RestaurantsDelivery': 'False', 'BusinessAcce...","Restaurants, Fast Food, Burgers","2011-08-18 23:18:26, 2011-09-20 16:38:54, 2011...",544
8,lK-wuiq8b1TuU7bfbQZgsg,Hingetown,,Cleveland,OH,44113,41.489343,-81.711029,3.0,4,"{'Alcohol': 'u'none'', 'GoodForKids': 'True', ...","Shopping Centers, Food, Coffee & Tea, Cafes, M...","2014-11-07 15:13:55, 2015-01-21 12:26:01, 2015...",1783
9,LAoSegVNU4wx4GTA8reB6A,Tzikii Food Truck,7510 S Priest Dr,Tempe,AZ,85283,33.350104,-111.963660,3.0,4,"{'RestaurantsTakeOut': 'False', 'WheelchairAcc...","Restaurants, Egyptian, Food Trucks, Food, Midd...",2018-05-24 02:18:53,19


In [79]:
restaurant_checkin_df.shape

(43039, 14)

In [89]:
restaurant_checkin_df.to_csv('restaurant_checkin.csv')

### 5) Tips

In [9]:
tip_json_path = 'yelp_dataset/yelp_academic_dataset_tip.json'
df_tip = pd.read_json(tip_json_path, lines=True)

In [10]:
df_tip.sample(6)

,user_id,business_id,text,date,compliment_count
1190328,nwQ_nIxVWYByTTqXiCQLAA,qihqxBCgXOPf2c-QxaFibA,Great service. \nGreat atmosphere.,2014-10-08 23:00:08,0
27680,HO3kgFkq7AMmkcE7QvUGLQ,blrsVPvhSZ2RPGnNuDfaxA,Open till 10pm!,2012-04-28 01:29:07,0
650835,Tx9YmzkGh2F-VT_c_Xcm2w,ezuzSOrWu3KhFKTtVAMWlQ,Iced americano is sweet!,2010-07-22 18:09:36,0
247579,SfY0jEFqBkrqq9bVi6P31Q,mh9_KtSt_ebl7vTKiy2vQw,"If you don't have a 4x4, park at Esselmont Par...",2015-01-01 16:45:13,0
483191,FeaOjUwDzX-AbDMcJk6lEg,stbX9rtRIpDF-VymOwHJ8w,Hoo-kaka,2010-11-07 03:59:01,0
517697,qmuCid52Y-SaJS_6v2CkNg,DlFKsdGY7-7c4u7_A-W99Q,Very drinkable latte but rather pricey,2015-12-11 19:04:30,0


In [11]:
df_tip.shape

(1320761, 5)

I didn't add up using the user tips for this project either, as I'm saving it for the NLP analysis.

**Glossary of df names:** <br>
* **df_business:** yelp's business.json. Contains business_id, name, geo info, stars, review count, attributes, categories and hours. Also exists as *'business_yelp.csv'*.
* **restaurant_df:** created by **me**, filtered for restaurants that are open. Contains same columns as business, minus the hours. Also exists in csv: *'restaurant_bus_yelp.csv'*.
* **user_df:** yelp's user.json. Not yet filtered for restaurants! Nearly 2 million users listed, so it will have to be merged with other df in order to distinguish restaurants. Also exists as *'user_yelp.csv'*.
* **review.json:** currently *work in progress*. Might not use for this project. It's nearly 6GB. Definitely needs to be reduced and normalized.
* **df_checkin:** yelp's checkin.json. With "total" column added.
* **restaurant_checkin_df:** Created by **me**, restaurant_df + df_checkin, merged on business_id. Also exists as *'restaurant_checkin.csv'*.
* **df_tip:** yelp's tip.json. Contains user_id, business_id, text of the tip, date	and compliment_count.

Going to merge df_user and tip_df on user_id:

* Code below exists mostly for the sake of testing out the read_csv function and make sure everything looks the way it should. **No particularly useful merges will be done below this line.** Merges performed on tip and restaurant data caused a lot of duplicate rows, because each restaurant has multiple tips from multiple users listed.

In [23]:
df_user = pd.read_csv("yelp_dataset/user_yelp.csv")

In [24]:
df_user.head()

,Unnamed: 0,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,0,ntlvfPzc8eglqvk92iDIAw,Rafael,553,2007-07-06 03:27:11,628,225,227,NaN,"oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg...",...,2,1,0,1,11,15,22,22,10,0
1,1,FOBRPlBHa3WPHFB5qYDlVg,Michelle,564,2008-04-28 01:29:25,790,316,400,"2008,2009,2010,2011,2012,2013","ly7EnE8leJmyqyePVYFlug, pRlR63iDytsnnniPb3AOug...",...,4,5,2,1,33,37,63,63,21,5
2,2,zZUnPeh2hEp0WydbAZEOOg,Martin,60,2008-08-28 23:40:05,151,125,103,2010,"Uwlk0txjQBPw_JhHsQnyeg, Ybxr1tSCkv3lYA0I1qmnPQ...",...,6,0,1,0,3,7,17,17,4,1
3,3,QaELAmRcDc5TfJEylaaP8g,John,206,2008-09-20 00:08:14,233,160,84,2009,"iog3Nyg1i4jeumiTVG_BSA, M92xWY2Vr9w0xoH8bPplfQ...",...,1,0,0,0,7,14,7,7,2,0
4,4,xvu8G900tezTzbbfqmTKvA,Anne,485,2008-08-09 00:30:27,1265,400,512,"2009,2010,2011,2012,2014,2015,2016,2017,2018","3W3ZMSthojCUirKEqAwGNw, eTIbuu23j9tOgmIa9POyLQ...",...,9,2,1,1,22,28,31,31,19,31


In [25]:
columns_to_drop = ['Unnamed: 0', 'friends', 'compliment_more', 'compliment_profile', 'compliment_cute', \
                  'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny',\
                   'compliment_writer', 'compliment_photos']
df_user = df_user.drop(columns_to_drop, axis = 1)


In [26]:
df_user.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,fans,average_stars,compliment_hot
0,ntlvfPzc8eglqvk92iDIAw,Rafael,553,2007-07-06 03:27:11,628,225,227,NaN,14,3.57,3
1,FOBRPlBHa3WPHFB5qYDlVg,Michelle,564,2008-04-28 01:29:25,790,316,400,"2008,2009,2010,2011,2012,2013",27,3.84,36
2,zZUnPeh2hEp0WydbAZEOOg,Martin,60,2008-08-28 23:40:05,151,125,103,2010,5,3.44,9
3,QaELAmRcDc5TfJEylaaP8g,John,206,2008-09-20 00:08:14,233,160,84,2009,6,3.08,2
4,xvu8G900tezTzbbfqmTKvA,Anne,485,2008-08-09 00:30:27,1265,400,512,"2009,2010,2011,2012,2014,2015,2016,2017,2018",78,4.37,8


In [28]:
df_user.shape

(1968703, 11)

What can I merge in order to link the users with restaurant businesses, in order to reduce the df_user and df_tip to just restaurant businesses? So far we have nearly 2 million users and 1.3 million tips written. <br>
Possibly, I could df_tip with restaurant_df on business_id.

In [39]:
df_restaurant_tip = pd.merge(restaurant_df, df_tip, on = 'business_id')

In [40]:
df_restaurant_tip.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,user_id,text,date,compliment_count
0,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...","Japanese, Fast Food, Food Court, Restaurants",kmzYKcAL2AGmAnlCF8UitA,The owners are really kind and cute,2018-07-24 21:10:31,0
1,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...","Japanese, Fast Food, Food Court, Restaurants",dSTRQSeCqMTbs7l8KF_xJg,If the roll you want isn't already made tell t...,2015-04-16 05:30:33,0
2,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...","Japanese, Fast Food, Food Court, Restaurants",jQngqbkDXAyDN_Z5Umc4KA,"Good taste, good price, friendly people!",2016-06-19 19:04:28,0
3,vjTVxnsQEZ34XjYNS-XUpA,Wetzel's Pretzels,"4550 East Cactus Rd, #KSFC-4",Phoenix,AZ,85032,33.602822,-111.983533,4.0,10,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...","Food, Pretzels, Bakeries, Fast Food, Restaurants",A0j21z2Q1HGic7jW6e9h7A,This Wetzel's Pretzels is located in the PV Fo...,2013-03-15 21:37:17,0
4,vjTVxnsQEZ34XjYNS-XUpA,Wetzel's Pretzels,"4550 East Cactus Rd, #KSFC-4",Phoenix,AZ,85032,33.602822,-111.983533,4.0,10,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...","Food, Pretzels, Bakeries, Fast Food, Restaurants",Q4Qfu-3vYtL1LRm2X1b0Gg,Jalapeno pretzel!,2012-12-21 17:37:21,0


In [42]:
df_restaurant_tip.shape

(695917, 16)

695,917 tips about 43,965 restaurants. That makes it over 100-150 tips per restaurant?

In [43]:
df_restaurant_tip.to_csv('restaurant_tip.csv')

Now, off to the Part 2: the **EDA notebook**.